In [4]:
from bs4 import BeautifulSoup as bs
import requests 
import pandas as pd
import re

                                            Задание 1
                                      Информация о вакансиях

In [5]:
details = {
    "Name":[],
    "Employer":[],
    "Minimum Salary":[],
    "Maximum Salary":[],
    "Currency":[],
    "Location":[],
    "Vacancy Link":[],
    "Site":[],
    
    
}

# pagecount = number of pages


def getAllPosts(position):
    pagecount = int(input("How many pages do you want ? "))
    totalpage = 0
    page = 0
   
    
    url=f"https://hh.ru/vacancies/{position}"
    array = []
    tag = "a"
    attr= "vacancy-serp-item"
    
    
    headers =  {
    'User-Agent':
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'
}
    while totalpage < pagecount:
        
        params={"page":page}


        request = requests.get(url,headers=headers,params=params)
        response = request.text

        soup = bs(response,"html.parser")


        nextPage = soup.find('a',attrs={"class":"bloko-button","href":f"/vacancies/{position}?page={page+1}","data-qa":"pager-next"})

        vacancies = soup.find_all("div",attrs={"class":"vacancy-serp-item"})

        for vac in vacancies:
            header = vac.find("a",attrs={"class":"bloko-link","data-qa":"vacancy-serp__vacancy-title"})
            sideBar = vac.find("span",attrs={"class":"bloko-header-section-3 bloko-header-section-3_lite","data-qa":"vacancy-serp__vacancy-compensation"})
            getName = header.text
            employer = vac.find("a",attrs={'class':"bloko-link bloko-link_secondary","data-qa":"vacancy-serp__vacancy-employer" }).text
            getLink = header['href']
            location = vac.find("span",attrs={"class":"vacancy-serp-item__meta-info","data-qa":"vacancy-serp__vacancy-address" }).text

            salary = sideBar.text if sideBar else ""
            currency = salary[-4:] if salary else None
           

            salary_array = salary.split("–") if salary else None
    
            if salary:
                if (len(salary_array) == 2):
                    minimum = int(('').join(re.findall(r'\d+',salary_array[0])))
                    maximum =int(('').join(re.findall(r'\d+',salary_array[1]))) 
                elif (len(salary_array) == 1):
                    maximum = int(('').join(re.findall(r'\d+',salary_array[0]))) if salary_array[0].find('до')==0 else None
                    minimum =int(('').join(re.findall(r'\d+',salary_array[0]))) if salary_array[0].find("от")==0 else None
            else:
                minimum = None
                maximum = None

            site = f"https://hh.ru/vacancies/{position}/?page={page}"
            details["Name"].append(getName)
            details["Employer"].append(employer)
            details["Minimum Salary"].append(minimum)
            details["Maximum Salary"].append(maximum)
            details["Location"].append(location)
            details["Vacancy Link"].append(getLink)
            details["Site"].append(site)
            details["Currency"].append(currency)
            
        
        print(f'{page+1}/{pagecount}')
        
        
        
        

        # Check for next page  


        if nextPage:
            proceed = True if nextPage.text == "дальше" else False
        else:
            proceed = False
            print('There are no more pages on this site')
            return


        if proceed and totalpage <= pagecount:
            page+=1
            totalpage+=1
    else:
        print(f"You have {pagecount} {f'pages' if pagecount>1 else f'page' }")
            



In [6]:
getAllPosts("english_teacher")

How many pages do you want ? 6
1/6
2/6
3/6
4/6
5/6
6/6
There are no more pages on this site


In [7]:
dataFrame = pd.DataFrame(details, columns=["Name","Employer","Minimum Salary","Maximum Salary","Currency","Location","Vacancy Link","Site"])

In [8]:
dataFrame

,Name,Employer,Minimum Salary,Maximum Salary,Currency,Location,Vacancy Link,Site
0,Преподаватель английского языка,Школа скорочтения и развития интеллекта IQ007 ...,40000.0,120000.0,руб.,"Москва, Бибирево",https://hh.ru/vacancy/46695251?from=catalog_va...,https://hh.ru/vacancies/english_teacher/?page=0
1,Preschool English Teacher / Воспитатель со зна...,ООО Bilingual Kids Academy,80000.0,100000.0,руб.,Москва,https://hh.ru/vacancy/46586552?from=catalog_va...,https://hh.ru/vacancies/english_teacher/?page=0
2,English Teacher (Native speaker),Детский сад Sun School Академика Туполева,60000.0,NaN,руб.,Москва,https://hh.ru/vacancy/46680146?from=catalog_va...,https://hh.ru/vacancies/english_teacher/?page=0
3,English Teacher (Native speaker),Sun School (ИП Щитова Елена Евгеньевна),100000.0,NaN,руб.,"Москва, Академическая",https://hh.ru/vacancy/46427689?from=catalog_va...,https://hh.ru/vacancies/english_teacher/?page=0
4,English Teacher (native speaker),Английский детский сад Sun School (ИП Николаев...,130000.0,NaN,руб.,"Москва, Баковка и еще 2",https://hh.ru/vacancy/46313252?from=catalog_va...,https://hh.ru/vacancies/english_teacher/?page=0
...,...,...,...,...,...,...,...,...
294,Proofreader at Practicum by Yandex,Яндекс.Практикум,NaN,NaN,None,Москва,https://hh.ru/vacancy/46013524?from=catalog_va...,https://hh.ru/vacancies/english_teacher/?page=5
295,Business development manager,Атос АйТи Солюшенс энд Сервисез,NaN,NaN,None,"Москва, Технопарк",https://hh.ru/vacancy/46210954?from=catalog_va...,https://hh.ru/vacancies/english_teacher/?page=5
296,Учитель начальных классов,Частная школа Взмах,70000.0,NaN,руб.,"Москва, Беляево",https://hh.ru/vacancy/46093575?from=catalog_va...,https://hh.ru/vacancies/english_teacher/?page=5
297,Консультант по ИБ,Positive Technologies,NaN,NaN,None,"Москва, Преображенская площадь",https://hh.ru/vacancy/46098240?from=catalog_va...,https://hh.ru/vacancies/english_teacher/?page=5


In [9]:
dataFrame.to_json("scrapped",force_ascii=False)
dataFrame.to_csv("scrapped_c")